# Building Data Lake for demographic analysis and prediction in the U.S.

## Data Engineering Capstone Project

### Project Summary
Main goal of the project is to building Data Lake for demographic analysis and prediction in the U.S.

There are several datasets with raw data:

* `I94 Immigration Data`;
* `U.S. City Demographic Data`;
* `Airport Codes`.

Project focuses on creating data model for the future analysis and building ETL-pipeline using Apache Spark. The whole ETL-pipeline processes raw data, cleanup, apply data quality checks and store to the analytical area in the Data Lake according to data model.
Resulting database allows analytics and data scientists to find and predict correlation between demographic situation in different cities/states and immigration to this locations.

The project follows the follow steps:

* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up11

In [57]:
# Do all imports and installs here
import re
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, Row, Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
import pyspark.sql.functions as F

In [4]:
# Set pandas options
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 500)

In [5]:
# Create and configure Spark session. Add support for sas7bdat data format.
spark = (
    SparkSession
    .builder
    .config('spark.jars.packages','saurfang:spark-sas7bdat:2.0.0-s_2.11')
    .config('spark.sql.session.timeZone', 'UTC')
    .enableHiveSupport()
    .getOrCreate()
)

### Step 1: Scope the Project and Gather Data

#### Scope

We will build a database (in the analytical area of our data lake) for demographic analysis and predictions which depends on immegration data.
Our resulting database will have the following structure:

* `cities` – dimension table with U.S. cities that will interested us in the future analytics. This table filled from the raw dataset represented by [us-cities-demographics.json](./us-cities-demographics.json) datasource file. We want to analyze only data for the same cities to compare with aggregated data for the 2015 year.
* `airports` – dimension table which contains data about U.S. international airports. This table filled from the raw dataset represented by [airport-codes_csv.csv](./airport-codes_csv.csv) data source file enriched with airport code in I94 data format (see explanation in the step 3). This table is not necessary for the solution, but `I94 Immigration Data` has own airport identifiers and we need to provide standard airport [IATA](https://en.wikipedia.org/wiki/IATA_airport_code) codes for our analysis team.
* `immigration` – fact table register immigration events.This table filled from the raw dataset represented by `I94 Immigration Data` SAS database joined with `airports` and `cities` data.
* `demographics` – pre-aggregated materialized view which is good candidate to land in data mart and contains aggregated data grouped by year and city. Each row of this table shows how much people (men, women and total) immigrated to the particular U.S. city each year. It can be used for the fast analytics and reporting. This table filled in two steps:
  1. Filled from the `us-cities-demographics.csv` datasource file which already contains data for the 2015 year.
  2. Filled from the `immigration` fact table by additition to the data for the previous years which allows us to see growth of the population.

##### Important notes and assumptions

1. We assume that all immigrants from the `I94 Immigration Data` will come and stay in the U.S. Actually we could analyze their visa types to be more precises, but for the scope of this project we will not consider it.
2. We assume that all immigrants will stay for living in the city to which the airport belongs. In real life this may not be the case, but we will not consider it.
3. Actually `demographics` table will contains data only for the 2015 and 2016 years because the `I94 Immigration Data` dataset contains data only for the 2016 year but our pipeliens will work for datasets with following years too.

##### Tools

We will use [Apache Spark](https://spark.apache.org/) to build our pipeline. Also we will use Data Lake idea to build our resulting tables and store our data in the HDFS or object storage like S3. We will not create staging area because in the Data Lake approache we can just load raw datasets with Spark, make all necessary transformations, cleanup the data, apply data quality checks and land it in the analytical area.

#### Describe and Gather Data

We will use several data sources to achieve our goal.

1. We will use `U.S. City Demographic Data` grouped by city for 2015 year provided by [U.S. Census Bureau](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/information/).
  * Data format: [JSON](https://en.wikipedia.org/wiki/JSON). You may see both JSON and CSV files with the same dataset in the repository, but we choose JSON format because we want to use different formats for each dataset.
  * Data source: [us-cities-demographics.json](./us-cities-demographics.json)
2. We will extract U.S. immigration data grouped by year, month and city from `I94 Immigration Data` provided by [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html).
  * Data format: [SAS7BDAT](https://cran.r-project.org/web/packages/sas7bdat/vignettes/sas7bdat.pdf)
  * Data format description: SAS labels and possible values and errors described in the [I94_SAS_Labels_Descriptions.SAS](./I94_SAS_Labels_Descriptions.SAS) file.
  * Data source: [i94_apr16_sub.sas7bdat](../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat). Actually there are bunch of files partitioned by month.
3. Also we will use [Airport Codes](https://datahub.io/core/airport-codes#data) table.
  * Data format: [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)
  * Data source: [airport-codes_csv.csv](./airport-codes_csv.csv)

#### U.S. City Demographic Data

Extract raw data from the data source and display few rows.

As you may see in the example below demogrphic data stored in the `fields` column. There are several fields which we have to extract to achieve our goal:

* city;
* state;
* state_code;
* male_population;
* female_population;
* total_population;

In [6]:
# Load raw data from the U.S. City Demographic Data and display first 5 rows
demographics_source_file_name = './us-cities-demographics.json'
demographics_demo_df = pd.read_json(demographics_source_file_name)
demographics_demo_df.head()

,datasetid,fields,record_timestamp,recordid
0,us-cities-demographics,"{'count': 2177650, 'city': 'Los Angeles', 'number_of_veterans': 85417, 'male_population': 1958998, 'foreign_born': 1485425, 'average_household_size': 2.86, 'median_age': 35.0, 'state': 'California', 'race': 'White', 'total_population': 3971896, 'state_code': 'CA', 'female_population': 2012898}",1970-01-01T03:00:00+03:00,7da42fda61238faccac3d43954a8f621a3a51194
1,us-cities-demographics,"{'count': 124270, 'city': 'Metairie', 'number_of_veterans': 7187, 'male_population': 69515, 'foreign_born': 19871, 'average_household_size': 2.39, 'median_age': 41.6, 'state': 'Louisiana', 'race': 'White', 'total_population': 146458, 'state_code': 'LA', 'female_population': 76943}",1970-01-01T03:00:00+03:00,f176fd28e69ee0c152db080858781e769840c6cc
2,us-cities-demographics,"{'count': 80781, 'city': 'Boca Raton', 'number_of_veterans': 4367, 'male_population': 44760, 'foreign_born': 21117, 'average_household_size': 2.22, 'median_age': 47.3, 'state': 'Florida', 'race': 'White', 'total_population': 93226, 'state_code': 'FL', 'female_population': 48466}",1970-01-01T03:00:00+03:00,ed4a94bf11b553ed8ebd93b7c24b0e8f326dadb0
3,us-cities-demographics,"{'count': 2566, 'city': 'Quincy', 'number_of_veterans': 4147, 'male_population': 44129, 'foreign_born': 32935, 'average_household_size': 2.39, 'median_age': 41.0, 'state': 'Massachusetts', 'race': 'Hispanic or Latino', 'total_population': 93629, 'state_code': 'MA', 'female_population': 49500}",1970-01-01T03:00:00+03:00,a4cd5f6782c79aa0348652718ca179ca390ce086
4,us-cities-demographics,"{'count': 16845, 'city': 'Union City', 'number_of_veterans': 1440, 'male_population': 38599, 'foreign_born': 32752, 'average_household_size': 3.46, 'median_age': 38.5, 'state': 'California', 'race': 'White', 'total_population': 74510, 'state_code': 'CA', 'female_population': 35911}",1970-01-01T03:00:00+03:00,b3b36950de605af92a78168f5127dff485a59b0b


#### I94 Immigration Data

Extract raw data from the data source and display few rows.

There are several fields which we have to extract to achieve our goal:

* `cicid` – unique row identifier;
* `i94yr` – year in 4 digit format;
* `i94mon` – month index (Jan = 1, Feb = 2, Mar = 3, etc.);
* `i94port` – airport code in 3 letters format described in the SAS file description;
* `i94mode` – border crossing method (Air = 1, Sea = 2, Land = 3, Not reported = 9);
* `i94addr` – U.S. state of arrival;
* `gender` – male (M) or female (F).

In [7]:
# Load raw data from the I94 Immigration Data and display first 5 rows
immigration_source_file_name = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_demo_df = pd.read_sas(immigration_source_file_name, 'sas7bdat', encoding='ISO-8859-1')
immigration_demo_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


#### Airport Codes

Extract raw data from the data source and display few rows.

There are several fields which we have to extract to achieve our goal:

* `iata_code` – unique [IATA](https://en.wikipedia.org/wiki/IATA_airport_code) airport code, we will extract only international airports and can use this code as identifier;
* `name` – airport name;
* `iso_country` – country, we will use this field to extract only U.S. airports;
* `iso_region` – country + state, we will use it to extract state;
* `municipality` – city to which the airport belongs.

In [12]:
# Load raw data from the Airport Codes and display first 5 rows
airports_source_file_name = './airport-codes_csv.csv'
airports_demo_df = pd.read_csv(airports_source_file_name, sep=',')
airports_demo_df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Step 2: Explore and Assess the Data

#### Explore the Data

For each of our data sources we will provide it's own cleaning steps.
Important part of this step that we just cleanup and filter raw data sets, we do not perform any joins between our data frames yet. It is preparation step to build our data model in following steps.

#### Cleaning Steps

1. Load and cleanup `U.S. City Demographic Data`
2. Load and cleanup `I94 Immigration Data`
  * Parse `I94_SAS_Labels_Descriptions.SAS`
  * Cleaup `I94 Immigration Data`
3. Load and cleanup `Airport Codes`

#### Load and cleanup U.S. City Demographic Data

`U.S. City Demographic Data` dataset is in JSON format. The essential data lives in the nested object, so we need extract it first.
Also we have to check that data in `male_population`, `fenale_population` and `total_population` columns are not empty.

In [43]:
def load_and_cleanup_demographic(source_filename):
    """
        Loads demographic data from the source file in JSON format
        and extract data from the inner JSON property.
        Apply cleanup steps:
        - male population is not empty;
        - female population is not empty;
        - total population is not empty.
    """

    # Load raw dataset as JSON using Spark
    raw_df = (
        spark
        .read
        .format('json')
        .load(source_filename)
    )

    # Actual data lives in the `fields` property, so we have to extract it.
    # Then apply cleanup steps.
    cleaned_df = (
        raw_df
        .select('fields.*')
        .filter(F.col('male_population').isNotNull())
        .filter(F.col('female_population').isNotNull())
        .filter(F.col('total_population').isNotNull())
    )
    return cleaned_df


# Uncomment following code to test load_and_cleanup_demographic function
# demographic_df = load_and_cleanup_demographic('./us-cities-demographics.json')
# demographic_df.show(5)

+----------------------+-----------+-------+-----------------+------------+---------------+----------+------------------+------------------+-------------+----------+----------------+
|average_household_size|       city|  count|female_population|foreign_born|male_population|median_age|number_of_veterans|              race|        state|state_code|total_population|
+----------------------+-----------+-------+-----------------+------------+---------------+----------+------------------+------------------+-------------+----------+----------------+
|                  2.86|Los Angeles|2177650|          2012898|     1485425|        1958998|      35.0|             85417|             White|   California|        CA|         3971896|
|                  2.39|   Metairie| 124270|            76943|       19871|          69515|      41.6|              7187|             White|    Louisiana|        LA|          146458|
|                  2.22| Boca Raton|  80781|            48466|       21117|          

#### Load and cleanup I94 Immigration Data

1. We have a lot of work to cleanup `I94 Immigration Data`. It is stored in SAS format and there is also exists `I94_SAS_Labels_Descriptions.SAS` file with description how to parse dataset values.
We need to write helper function which can map dataset columns values to real values using this description. This is not a cleanup step, but it is important preparation step for the future work.

2. Also we have to cleanup entire `I94 Immigration Data` dataset:

  * `i94port` should be in the list of valid airports described in the `I94_SAS_Labels_Descriptions.SAS` file;
  * `i94mode` = 1, we are interested in only in people who arrived to the U.S. by air, because only for them we can determine destination city;
  * `gender` is not empty, because we want to calculate statistics by men and women separately;
  * we should exract U.S. city and U.S. state from the `i94port` using SAS descriptions metadata;
  * also we should apply data type casts for some columns.

In [103]:
class I94Airport:
    """
        I94Airport contains parsed information from `i94port` field
        of the `I94 Immigration Data`.
        Also it implements static method `sas_description_parse_airport_codes`
        which parses SAS descriptions and returns dictionary
        with airport codes as keys and I94Aiport objects as values.
    """

    def __init__(self, airport_code, us_city, us_state):
        self._airport_code = airport_code
        self._us_city = us_city
        self._us_state = us_state

    def __str__(self):
        return f'{self._us_city}, {self._us_state} [{self._airport_code}]'

    @property
    def airport_code(self):
        return self._airport_code

    @property
    def us_city(self):
        return self._us_city

    @property
    def us_state(self):
        return self._us_state

    @staticmethod
    def sas_description_parse_airport_codes(sas_description_filename):
        """
            Load SAS description file, parse `i94port` values,
            filter non-U.S., "Collapsed" or "No port" data
            and returns dictionary with parsed results to fast lookups.
        """

        airports = {}
        is_airports_section = False

        # We will use regex to extract all three parts (code, city and state) from the description file.
        # Example: 'LOS' = 'LOS ANGELES, CA       '
        # Regex pattern matching: LOS = match[0], LOS ANGELES = match[1], CA = match[2]
        # Also current structure of regex eliminates all invalid airport codes.
        airport_regex = re.compile(r'\'([A-Z0-9]{3})\'.*=.*\'([A-Z-\/,\.\ ]+),\ ?([A-Z]{2})\ *\'')

        with open(sas_description_filename) as f:
            for line in f:
                line = line.strip()
                if line == ';':
                    is_airports_section = False
                elif line == 'value $i94prtl':
                    is_airports_section = True
                elif is_airports_section:
                    airport_parts = airport_regex.findall(line)
                    if airport_parts:
                        airport_code, us_city, us_state = airport_parts[0]
                        airports[airport_code] = I94Airport(airport_code, us_city, us_state)
        return airports


# Uncomment following code to test class I94Airport and it's static method sas_description_parse_airport_codes
# i94_airports = I94Airport.sas_description_parse_airport_codes('./I94_SAS_Labels_Descriptions.SAS')
# i94_airports_df = pd.DataFrame.from_dict(i94_airports, orient='index', columns=['I94 Airport'])
# i94_airports_df.head()

,I94 Airport
ALC,"ALCAN, AK [ALC]"
ANC,"ANCHORAGE, AK [ANC]"
BAR,"BAKER AAF - BAKER ISLAND, AK [BAR]"
DAC,"DALTONS CACHE, AK [DAC]"
PIZ,"DEW STATION PT LAY DEW, AK [PIZ]"


In [9]:
def load_and_cleanup_i94_immigration(sas_source_filename, sas_description_filename):
    """
        Loads and cleanup I94 Immigration Data.
        Apply cleanup steps:
        - i94port should be in the list of valid airports;
        - i94mode = 1 (arrived to the U.S. by air);
        - gender is not empty.
    """

    # Load airports from SAS description file
    i94_airports = I94Airport.sas_description_parse_airport_codes(sas_description_filename)

    # Load raw dataset as SAS using Spark
    raw_df = (
        spark
        .read
        .format('com.github.saurfang.sas.spark')
        .load(sas_source_filename)
    )

    # Cleanup data and repartition by `i94port` to speedup future queries
    cleaned_df = (
        raw_df
        .filter(F.col('i94mode') == 1)
        .filter(F.col('gender').isNotNull())
        .filter(F.col('i94port').isin(list(i94_airports.keys())))
        .withColumn('cicid_long', F.col('cicid').cast(LongType()))
        .withColumn('i94yr_int', F.col('i94yr').cast(IntegerType()))
        .withColumn('i94mon_int', F.col('i94mon').cast(IntegerType()))
        .repartition('i94port')
    )
    return cleaned_df


# Uncomment following code to test load_and_cleanup_i94_immigration function
immigration_df = load_and_cleanup_i94_immigration('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', './I94_SAS_Labels_Descriptions.SAS')
immigration_df.show(5)

+-------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+----------+---------+----------+
|  cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|cicid_long|i94yr_int|i94mon_int|
+-------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+----------+---------+----------+
| 2302.0|2016.0|   4.0| 107.0| 107.0|    FMY|20545.0|    1.0|   null|20558.0|  64.0|    1.0|  1.0|20160401|    null| null|      T|      K|   null|      M| 1952.0|09302016|     F|  null|     SY|   6.66572685E8|  514|      B1|     

#### Load and cleanup Airport Codes

There are several steps to cleanup airport codes data:

* we are interested only in the U.S. airports, thus we will preserve data by a condition `iso_country` = `US`;
* we need only international airports, thus we will filter out all data with empty `iata_code`;
* we need only airports belongs to cities (filled `municipality` field);
* we need to extract U.S. state code from `iso_region` field, which contains state prefixed by ISO country code.

In [102]:
def load_and_cleanup_airport_codes(source_filename):
    """
        Loads airport codes data from the source file in CSV format.
        Apply cleanup steps:
        - preserve only U.S. airport codes;
        - remove all rows with empty IATA code;
        - remove all rows with empty city (municipality);
        - add column `state_code` with extracted U.S. state from the `iso_region` field.
    """

    # Load raw dataset as CSV using Spark
    raw_df = (
        spark
        .read
        .format('csv')
        .option('header', 'true')
        .option('delimiter', ',')
        .load(source_filename)
    )

    # Cleanup data
    cleaned_df = (
        raw_df
        .filter(F.col('iso_country') == 'US')
        .filter(F.col('iata_code').isNotNull())
        .filter(F.col('municipality').isNotNull())
        .withColumn('state_code', F.col('iso_region')[-2:2])
    )
    return cleaned_df


# Uncomment following code to test load_and_cleanup_demographic function
# airport_codes_df = load_and_cleanup_airport_codes('./airport-codes_csv.csv')
# airport_codes_df.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+----------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region| municipality|gps_code|iata_code|local_code|         coordinates|state_code|
+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+----------+
| 07FA|small_airport|Ocean Reef Club A...|           8|       NA|         US|     US-FL|    Key Largo|    07FA|      OCA|      07FA|-80.274803161621,...|        FL|
|  0AK|small_airport|Pilot Station Air...|         305|       NA|         US|     US-AK|Pilot Station|    null|      PQS|       0AK|-162.899994, 61.9...|        AK|
| 0CO2|small_airport|Crested Butte Air...|        8980|       NA|         US|     US-CO|Crested Butte|    0CO2|      CSE|      0CO2|-106.928341, 38.8...|        CO|
| 0TE7|sma

### Step 3: Define the Data Model

#### 3.1 Conceptual Data Model

Currently analytical area in the Data Lake consists of two dimension tables, one fact table and one aggregated table (materialized view) which could be considered as a part of Data Mart.

1. `cities` dimension table contains information about U.S. cities. It is filled from the `U.S. City Demographic Data` data source.
  Table structure:
  * `city_id` – auto-generated surrogate row identifier;
  * `name` – U.S. city name;
  * `state_name` – U.S. state name;
  * `state_code` – U.S. state code.

  `cities` dimension table will be partitioned by `state_code` because it is how it will usually queried.


2. `airports` dimension table contains information about U.S. international airports. This table is filled from the `Airport Codes` and actually used to link U.S. cities with the immigration data via the airport code. So here we will use the trick and fill this table from the `Airport Codes` joined with SAS Descripton file for the `I94 Immigration Data` which is much easier then parse whole `I94 Immigration Data` dataset. Also this trick will speedup future steps.
  Table structure:
  * `airport_id` – auto-generated surrogate row identifier. Actually we could use iata_code as identifier, but we will use numeric IDs for consistency and extensibility, because in future we could add local airports too for some reason;
  * `name` – airport name;
  * `city` – airport city (could be used to join with `cities`);
  * `state_code` – U.S. state code (could be used to join with `cities`);
  * `iata_code` – IATA airport code;
  * `i94_code` – airport code in I94 data format.

  `airports` dimension table will be partitioned by `state_code` for future joins with `cities`.


3. `immigration` fact table register immigration events which come from the `I94 Immigration Data` data source. Also it will join information from the `airports` and `cities` to link each immigration event with correct `city_id`.

  Table structure:
  * `immigration_id` – auto-generated surrogate row identifier;
  * `city_id` – reference key to the `cities`.`city_id`;
  * `airport_id` – reference key to the `airports`.`airport_id`;
  * `year` – 4-digit year;
  * `month` – 1-digit month (1 - Jan, 2 - Feb, 3 - Mar, etc.);
  * `gender` – 1-character gender (M - Male, F - Female);
  * `i94_cicid` – identifier from the `I94 Immigration Data` to allow find and check row in the original data source. Also in could help in the events deduplication procedure;
  * `city` – destination U.S. city name (add redundancy to facilitate analytics);
  * `state_code` – destination U.S. state code (add redundancy to facilitate analytics);
  * `iata_code` – destination IATA airport code (add redundancy to facilitate analytics).

  `immigration` fact table will be partitioned by `year` and `city_id` because we will group by fields to fill `demographics` table.


4. `demographics` materialized view contains aggregated data from the `immigration` table grouped by year and city. This table filled from the two data sources: `U.S. City Demographic Data` which contains already pre-calculated data for 2015 year and from the `immigration` fact table joined with `cities`.

  Table structure:
  * `demographics_id` – auto-generated surrogate row identifier;
  * `city_id` – reference key to the `cities`.`city_id`;
  * `year` – 4-digit year;
  * `city` – U.S. city name (add redundancy to facilitate analytics);
  * `state_code` – U.S. state code (add redundancy to facilitate analytics);
  * `male_population` – male population amount;
  * `female_population` – female population amount;
  * `total_population` – total population amount.

  `demographics` materialized view will be partitioned by `year` and `state_code` because usually to speedup most frequently queries.

The overall analytical area database diagram is shown on the following scheme:

![capstone_schema](images/capstone_schema.png)


#### 3.2 Mapping Out Data Pipelines

The entire ETL-pipeline contains the following steps:
1. Load and cleanup the `U.S. City Demographic Data` (see step 2).
2. Load and cleanup the `Airport Codes Data` (see step 2).
3. Load and cleanup the `I94 immigration Data` (see step 2).
4. Create schema for the `cities` dimension table.
5. Fill `cities` dimension table with data and write to the analytical area as a `parquet` file format partitioned by `state_code`.
6. Create schema for the `airports` dimension table.
7. Fill `airports` dimension table with data and write to the analytical area as a `parquet` file format partitioned by `state_code`.
8. Create schema for the `immigration` fact table.
9. Fill `immigration` fact table with data and write to the analytical area as a `parquet` file format partitioned by `year` and `city_id`.
10. Create schema for the `demographics` materialized view.
11. Fill `demographics` materialized view with data and write to the analytical area as a `parquet` file format partitioned by `year` and `state_code`.

### Step 4: Run Pipelines to Model the Data
#### 4.1 Create the data model
Build the data pipelines to create the data model.

##### ETL-pipeline steps [1-3]: Load and cleanup all raw data

In [13]:
# Set paths to data sources
demographics_filename = './us-cities-demographics.json'
airport_codes_filename = './airport-codes_csv.csv'
# immigration_sas_filename = '../../data/18-83510-I94-Data-2016/*.sas7bdat'
immigration_sas_filename = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_sas_description_filename = './I94_SAS_Labels_Descriptions.SAS'

# Load and cleanup data
demographic_df = load_and_cleanup_demographic(demographics_filename)
airport_codes_df = load_and_cleanup_airport_codes(airport_codes_filename)
immigration_df = load_and_cleanup_i94_immigration(immigration_sas_filename, immigration_sas_description_filename)

##### ETL-pipeline steps [4-5]: Fill `cities` dimension table

In [15]:
# Create schema for `cities` dimension table to validate data correctness
cities_schema = StructType([
    StructField('city_id', LongType(), nullable=False),
    StructField('name', StringType(), nullable=False),
    StructField('state_name', StringType(), nullable=False),
    StructField('state_code', StringType(), nullable=False)
])

# Create rdd from the `demographic_df` dataframe selecting required fields
cities_rdd = (
    demographic_df
    .withColumn('city_id', F.monotonically_increasing_id() + 1)
    .dropDuplicates(['city', 'state_code'])
    .select('city_id', 'city', 'state', 'state_code')
    .rdd
)

# Create `cities` table using prepared data and schema
cities_table = spark.createDataFrame(cities_rdd, cities_schema)

# Write `cities` dimension table to parquet partitioned by `state_code`
cities_table_path = './tables/cities/cities.parquet'
(
    cities_table
    .write
    .partitionBy('state_code')
    .mode('overwrite')
    .parquet(cities_table_path)
)

##### ETL-pipeline steps [6-7]: Fill `airports` dimension table

In [16]:
# Create schema for `airports` dimension table to validate data correctness
airports_schema = StructType([
    StructField('airport_id', LongType(), nullable=False),
    StructField('name', StringType(), nullable=False),
    StructField('city', StringType(), nullable=False),
    StructField('state_code', StringType(), nullable=False),
    StructField('iata_code', StringType(), nullable=False),
    StructField('i94_code', StringType(), nullable=True)
])

# Construct dataframe with airport information using SAS description for the `I94 Immigration Data`
# using I94Airport class described above (see step 2)
i94_airports = I94Airport.sas_description_parse_airport_codes(immigration_sas_description_filename)
i94_airports_df = (
    spark
    .sparkContext
    .parallelize(list(i94_airports.keys()))
    .map(lambda i94port: Row(**{
        'i94_port': i94port.upper(),
        'i94_city': i94_airports[i94port].us_city.upper(),
        'i94_state_code': i94_airports[i94port].us_state.upper(),
    }))
    .toDF()
)


# Create rdd from the `airport_codes_df` dataframe joined with `i94_airports_df` dataframe
airports_rdd = (
    airport_codes_df
    .withColumn('airport_id', F.monotonically_increasing_id() + 1)
    .withColumn('city_upper', F.upper(F.col('municipality')))
    .join(i94_airports_df,
          (F.col('state_code') == i94_airports_df.i94_state_code)
          & (F.col('city_upper') == i94_airports_df.i94_city)
          , 'left_outer')
    .dropDuplicates(['iata_code'])
    .select('airport_id', 'name', 'municipality', 'state_code', 'iata_code', 'i94_port')
    .rdd
)

# Create `airports` table using prepared data and schema
airports_table = spark.createDataFrame(airports_rdd, airports_schema)

# Write `cities` dimension table to parquet partitioned by `state_code`
airports_table_path = './tables/airports/airports.parquet'
(
    airports_table
    .write
    .partitionBy('state_code')
    .mode('overwrite')
    .parquet(airports_table_path)
)

##### ETL-pipeline steps [8-9]: Fill `immigration` fact table

In [ ]:
# Create schema for `immigration` fact table to validate data correctness
immigration_schema = StructType([
    StructField('immigration_id', LongType(), nullable=False),
    StructField('city_id', LongType(), nullable=False),
    StructField('airport_id', LongType(), nullable=False),
    StructField('year', IntegerType(), nullable=False),
    StructField('month', IntegerType(), nullable=False),
    StructField('gender', StringType(), nullable=False),
    StructField('i94_cicid', LongType(), nullable=False),
    StructField('city', StringType(), nullable=False),
    StructField('state_code', StringType(), nullable=False),
    StructField('iata_code', StringType(), nullable=True)
])

# Create rdd from the `immigration_df` dataframe joined with already created airports_table and cities_table dataframes
immigration_rdd = (
    immigration_df
    .join(airports_table,
          (immigration_df.i94port == airports_table.i94_code)
          , 'inner')
    .join(cities_table,
          (cities_table.name == airports_table.city)
          & (cities_table.state_code == airports_table.state_code)
          , 'inner')
    .withColumn('immigration_id', F.monotonically_increasing_id() + 1)
    .dropDuplicates(['cicid'])
    .select(
        F.col('immigration_id'),
        cities_table.city_id.alias('city_id'),
        airports_table.airport_id.alias('airport_id'),
        immigration_df.i94yr_int.alias('year'),
        immigration_df.i94mon_int.alias('month'),
        immigration_df.gender.alias('gender'),
        immigration_df.cicid_long.alias('i94_cicid'),
        cities_table.name.alias('city'),
        cities_table.state_code.alias('state_code'),
        airports_table.iata_code.alias('iata_code')
    )
    .rdd
)

# Create `immigration` table using prepared data and schema
immigration_table = spark.createDataFrame(immigration_rdd, immigration_schema)

Write `immigration` fact table to parquet partitioned by `year` and `city_id`
immigration_table_path = './tables/immigration/immigration.parquet'
(
    immigration_table
    .write
    .partitionBy('year', 'city_id')
    # .mode('append') for the sake of the demo capstone project we will use mode=overwrite to save storage space
    .mode('overwrite')
    .parquet(immigration_table_path)
)

##### ETL-pipeline steps [10-11]: Fill demographics materialized view¶

In [45]:
# Create schema for `demographics` table to validate data correctness
demographics_schema = StructType([
    StructField('demographics_id', LongType(), nullable=False),
    StructField('city_id', LongType(), nullable=False),
    StructField('year', IntegerType(), nullable=False),
    StructField('city', StringType(), nullable=False),
    StructField('state_code', StringType(), nullable=False),
    StructField('male_population', LongType(), nullable=False),
    StructField('female_population', LongType(), nullable=False),
    StructField('total_population', LongType(), nullable=False)
])

# 1. We need to load data for 2015 year from the U.S. City Demographic Data
# as a basis for the future years cumulative data.
# For this task we have to join `demographic_df` with `cities_table`
# and group by data by `state_code` and `city`.
demographics_2015_df = (
    demographic_df
        .join(cities_table,
          (cities_table.name == demographic_df.city)
          & (cities_table.state_code == demographic_df.state_code)
          , 'inner')
    .groupBy([demographic_df.state_code, demographic_df.city, cities_table.city_id])
    .agg({'male_population': 'sum', 'female_population': 'sum', 'total_population': 'sum'})
    .withColumn('year', F.lit(2015))
    .select(
        cities_table.city_id.alias('city_id'),
        F.col('year'),
        demographic_df.city.alias('city'),
        demographic_df.state_code.alias('state_code'),
        F.col('sum(male_population)').alias('male_population'),
        F.col('sum(female_population)').alias('female_population'),
        F.col('sum(total_population)').alias('total_population')
    )
)

# 2. Also we need to load data from the `immigration_table` grouped by `year`, `state_code` and `city`.
demographics_next_df = (
    immigration_table
    .withColumn('male_population', F.when(F.col('gender') == 'M', 1).otherwise(0))
    .withColumn('female_population', F.when(F.col('gender') == 'F', 1).otherwise(0))
    .withColumn('total_population', F.lit(1))
    .groupBy('year', 'state_code', 'city', 'city_id')
    .agg({'male_population': 'sum', 'female_population': 'sum', 'total_population': 'sum'})
    .select(
        'city_id',
        'year',
        'city',
        'state_code',
        F.col('sum(male_population)').alias('male_population'),
        F.col('sum(female_population)').alias('female_population'),
        F.col('sum(total_population)').alias('total_population')
    )
)

# 3. Finally we should combine two previous results to one dataset.
# The tricky part here is to add results of the previous year to the next year,
# in other words we want to get cumulative results each next year depending on previous year value.
# We use Window functions to achive this goal.
demographics_window = (
    Window
    .partitionBy('city_id')
    .orderBy(F.asc('year'))
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

demographics_rdd = (
    demographics_2015_df
    .unionAll(demographics_next_df)
    .withColumn('cumulative_male_population', F.sum('male_population').over(demographics_window))
    .withColumn('cumulative_female_population', F.sum('female_population').over(demographics_window))
    .withColumn('cumulative_total_population', F.sum('total_population').over(demographics_window))
    .withColumn('demographics_id', F.monotonically_increasing_id() + 1)
    .select(
        'demographics_id',
        'city_id',
        'year',
        'city',
        'state_code',
        'cumulative_male_population',
        'cumulative_female_population',
        'cumulative_total_population'
    )
    .rdd
)

# Create `demographics` table using prepared data and schema
demographics_table = spark.createDataFrame(demographics_rdd, demographics_schema)

# Write `demographics` table to parquet partitioned by `year` and `state_code`
demographics_table_path = './tables/demographics/demographics.parquet'
(
    demographics_table
    .write
    .partitionBy('year', 'state_code')
    .mode('overwrite')
    .parquet(demographics_table_path)
)

#### 4.2 Data Quality Checks
There are two types of data quality checks:

1. Each table should not be empty.
2. We will test `demographics` table for a few major U.S. city for 2015 and 2016 years. We expect that population in 2016 should be higher than in 2015. It is because that our project counts only immigrations to the U.S. and does not take into account any other population changes, thus population can only growth.

##### Perform data quality checks

In [97]:
def table_is_not_empty_check(table_df, table_name):
    """Data quality check. It tests that table is not empty."""

    print(f'Performs data quality check. Tests table "{table_name}" to be not empty...')

    row_count = table_df.count()
    assert row_count > 0, f'Data quality check failed. Table "{table_name}" is empty.'

    print(f'Data quality check passed. Table "{table_name}" has {row_count} rows.')


def population_changes_check(demographics_table_df, demographics_table_name):
    """
        Data quality check.
        It takes some major U.S. cities and compare population in 2015 and 2016 years.
        Population in 2016 should be greater for each city, because the only population change
        that we take into account is immigration to the U.S.
        Also we expect that in major cities at least few peoples will come each year.
    """

    print(f'Performs data quality check. Tests demographics table "{demographics_table_name}" that population should grows...')

    cities = ['Los Angeles, CA', 'New York, NY', 'San Francisco, CA', 'Washington, DC']

    demographics_pd_df = (
        demographics_table_df
        .filter(F.col('year').isin([2015, 2016]))
        .withColumn('city_with_state', F.concat_ws(', ', F.col('city'), F.col('state_code')))
        .filter(F.col('city_with_state').isin(cities))
        .select('year', 'city_with_state', 'total_population')
        .toPandas()
    )

    demographics_2015_pd_df = demographics_pd_df[demographics_pd_df['year'] == 2015].copy()
    demographics_2016_pd_df = demographics_pd_df[demographics_pd_df['year'] == 2016].copy()
    demographics_2016_pd_df['diff'] = demographics_2016_pd_df['total_population'] - demographics_2015_pd_df['total_population'].values

    for _, row in demographics_2016_pd_df.iterrows():
        city = row['city_with_state']
        diff = row['diff']
        assert diff > 0, \
            f'Data quality check failed. Table "{demographics_table_name}" has negative population changes between 2015 and 2016 for "{city}".'

    print(f'Data quality check passed. Table "{demographics_table_name}" has positive population changes between 2015 and 2016 years.')


# Perform data quality checks
table_is_not_empty_check(cities_table, 'cities')
table_is_not_empty_check(airports_table, 'airports')
table_is_not_empty_check(immigration_table, 'immigration')
table_is_not_empty_check(demographics_table, 'demographics')
population_changes_check(demographics_table, 'demographics')

Performs data quality check. Tests table "cities" to be not empty...
Data quality check passed. Table "cities" has 596 rows.
Performs data quality check. Tests table "airports" to be not empty...
Data quality check passed. Table "airports" has 2011 rows.
Performs data quality check. Tests table "immigration" to be not empty...
Data quality check passed. Table "immigration" has 2053055 rows.
Performs data quality check. Tests table "demographics" to be not empty...
Data quality check passed. Table "demographics" has 661 rows.
Performs data quality check. Tests demographics table "demographics" that population should grows...
Data quality check passed. Table "demographics" has positive population changes between 2015 and 2016 years.


#### 4.3 Data dictionary

Data model consists of four tables:

* `cities` – dimension table which contains information about U.S. cities;
* `airports` – dimension table which contains information about U.S. airports;
* `immigration` – fact table which register facts about immigration to U.S.;
* `demographics` – materialized view which describe population changes in the U.S. cities.

Actually data dictionary was already partially described in the step `3.1. Conceptual Data Model` (see step 3.1). Thus here we will provide summary on each table and each field.

##### Data dictionary for `cities`

`cities` dimension table contains information about U.S. cities. It is filled from the `U.S. City Demographic Data` data source.

Table structure:
  * `city_id` – auto-generated surrogate row identifier. It is easier for practice usage to reference by numeric ID;
  * `name` – U.S. city name;
  * `state_name` – U.S. state name;
  * `state_code` – U.S. state code.

The main table purpose is to keep up to date dictionary of the U.S. cities for which we want analyze demogrphics situation.

Print a few rows from `cities` dimension table to show the data in the resulting database:

In [99]:
# `citites` data example
airports_table.show(5)

+----------+--------------------+----------+----------+---------+--------+
|airport_id|                name|      city|state_code|iata_code|i94_code|
+----------+--------------------+----------+----------+---------+--------+
|       331|Greater Binghamto...|Binghamton|        NY|      BGM|    null|
|        20|      Eagle Air Park|  Brazoria|        TX|      BZT|    null|
|       488|Chanute Martin Jo...|   Chanute|        KS|      CNU|    null|
|       510|C David Campbell ...| Corsicana|        TX|      CRS|    null|
|       715|          Page Field|Fort Myers|        FL|      FMY|     FMY|
+----------+--------------------+----------+----------+---------+--------+
only showing top 5 rows



##### Data dictionary for `airports`

`airports` dimension table contains information about U.S. international airports. This table is filled from the `Airport Codes` and actually used to link U.S. cities with the immigration data via the airport code.

Table structure:
* `airport_id` – auto-generated surrogate row identifier. Actually we could use iata_code as identifier, but we will use numeric IDs for consistency and extensibility, because in future we could add local airports too for some reason;
* `name` – airport name;
* `city` – airport city;
* `state_code` – U.S. state code;
* `iata_code` – IATA airport code;
* `i94_code` – airport code in I94 data format.

The main table purpose is to keep up to date dictionary of the U.S. international airports. It will be used to link immigration data with destination cities.

Print a few rows from `airports` dimension table to show the data in the resulting database:

In [98]:
# `airports` data example
airports_table.show(5)

+----------+--------------------+----------+----------+---------+--------+
|airport_id|                name|      city|state_code|iata_code|i94_code|
+----------+--------------------+----------+----------+---------+--------+
|       331|Greater Binghamto...|Binghamton|        NY|      BGM|    null|
|        20|      Eagle Air Park|  Brazoria|        TX|      BZT|    null|
|       488|Chanute Martin Jo...|   Chanute|        KS|      CNU|    null|
|       510|C David Campbell ...| Corsicana|        TX|      CRS|    null|
|       715|          Page Field|Fort Myers|        FL|      FMY|     FMY|
+----------+--------------------+----------+----------+---------+--------+
only showing top 5 rows



##### Data dictionary for `immigration`

`immigration` fact table register immigration events which come from the `I94 Immigration Data` data source. Also it will join information from the `airports` and `cities` to link each immigration event with correct city.

Table structure:
* `immigration_id` – auto-generated surrogate row identifier;
* `city_id` – reference key to the `cities`.`city_id`;
* `airport_id` – reference key to the `airports`.`airport_id`;
* `year` – 4-digit year;
* `month` – 1-digit month (1 - Jan, 2 - Feb, 3 - Mar, etc.);
* `gender` – 1-character gender (M - Male, F - Female);
* `i94_cicid` – identifier from the `I94 Immigration Data` to allow find and check row in the original data source. Also in could help in the events deduplication procedure;
* `city` – destination U.S. city name;
* `state_code` – destination U.S. state code;
* `iata_code` – destination IATA airport code.

The core table in our database. Actual immigration events are registered in this table.

Print a few rows from `immigration` fact table to show the data in the resulting database:

In [100]:
# `immigration` data example
immigration_table.show(5)

+--------------+-------+----------+----+-----+------+---------+-------------+----------+---------+
|immigration_id|city_id|airport_id|year|month|gender|i94_cicid|         city|state_code|iata_code|
+--------------+-------+----------+----+-----+------+---------+-------------+----------+---------+
|  274877906948|    472|      1484|2016|    4|     M|      558|San Francisco|        CA|      SFO|
|  558345748553|    119|      1987|2016|    4|     M|     2862|       Boston|        MA|      BNH|
| 1357209665577|    273|       477|2016|    4|     F|     3901|    Charlotte|        NC|      CLT|
| 1022202221979|    701|       144|2016|    4|     F|     4066|     New York|        NY|      JRB|
| 1022202222917|    701|       144|2016|    4|     F|     5858|     New York|        NY|      JRB|
+--------------+-------+----------+----+-----+------+---------+-------------+----------+---------+
only showing top 5 rows



##### Data dictionary for `demographics`

`demographics` materialized view contains aggregated data from the `immigration` table grouped by year and city. This table filled from the two data sources: `U.S. City Demographic Data` which contains already pre-calculated data for 2015 year and from the `immigration` fact table joined with `cities`.

Table structure:
* `demographics_id` – auto-generated surrogate row identifier;
* `city_id` – reference key to the `cities`.`city_id`;
* `year` – 4-digit year;
* `city` – U.S. city name (add redundancy to facilitate analytics);
* `state_code` – U.S. state code (add redundancy to facilitate analytics);
* `male_population` – male population amount;
* `female_population` – female population amount;
* `total_population` – total population amount.

Aggreagate materialized view depends on immigration data. It is used for analytics and reporting.

Print a few rows from `demographics` table to show the data in the resulting database:

In [101]:
# `demographics` data example
demographics_table.show(5)

+---------------+-------+----+----------+----------+---------------+-----------------+----------------+
|demographics_id|city_id|year|      city|state_code|male_population|female_population|total_population|
+---------------+-------+----+----------+----------+---------------+-----------------+----------------+
|              1|     26|2015|Cedar Park|        TX|         156240|           167945|          324185|
|              2|     29|2015|   Fremont|        CA|         571915|           589040|         1160955|
|     8589934593|     65|2015|Greensboro|        NC|         661255|           765465|         1426720|
|     8589934594|    191|2015|     Tampa|        FL|         877585|           967555|         1845140|
|     8589934595|    191|2016|     Tampa|        FL|         886986|           977231|         1864221|
+---------------+-------+----+----------+----------+---------------+-----------------+----------------+
only showing top 5 rows



#### Step 5: Complete Project Write Up

Q: **Clearly state the rationale for the choice of tools and technologies for the project.**
A: The core descision is to build Data Lake architecure and to store resulting database in the Data Lake as a parquet files. It allows us to keep raw and prepared for analytics data in one place, also it allows to our analytics team to search insights in the raw data and get benfits from both raw and pre-aggregated data too. As a tool to process raw data and build resulting database was choosen [Apache Spark](https://spark.apache.org/). It is powerful tool that allows us to read with different data formats, loads, transform and aggregate data. The very important that Apache Spark uses lazy evaluation which allows us express complicated transformations in multiple steps and also get benefits from Catalyst optimizer and execute it in the entire data collection operation.

Q: **Propose how often the data should be updated and why.**
A: It depends of our goals. If we mostly want to work with `demographics` table which contains data partitioned by year we could update it yearly.
But here we can have one problem: if raw immigration data comes very frequently then yearly pipeline means processing very large amount of data which can take ages. Maybe we should consider to use stream or micro-batch processing to fill `immigration` table and then update `demographics` using batches at discrete points in time more frequently.


#### Write a description of how you would approach the problem differently under the following scenarios

Q: **The data was increased by 100x.**
A: In this case we possibly should replace batch processing to stream processing using Spark Streaming or other tools. Thus we can update our `immigration` fact table nearly real-time. Also we should update `demographics` data more frequently. For example we can update it monthly. It means that we will read `immigration` data only for the last month which reduce amount of processed data in one batch. Also we wil update `demographics` by adding month by month data.

Q: **The data populates a dashboard that must be updated on a daily basis by 7am every day.**
A: Actually if we want to populate a dashboard and we want to see changes each day, then we need some kind of orchestration tool to run our ETL-pipeline by schedule. For example [Apache Airflow](https://airflow.apache.org/) is a great tool for this purpose which allows us to run our pipeline daily.

Q: **The database needed to be accessed by 100+ people.**
A: First what we can do in this situation is to store our parquet files in the HDFS and give read access to our users. Also, if we have a lot of analytics who want to work only with aggragated data than we can create Data Mart for them, maybe to take some relational database (for example MS SQL Server) and load our `demographics` table into it.